# Torch
## Check GPU

In [1]:
import torch
import numpy as np
import sys
sys.path.append('..')

from torchlib.utils import list_device,set_device

# S1: check GPU
#list_device()

# S2: default parameters
set_device(1)
np.set_printoptions(precision = 2)
torch.set_default_dtype(torch.float32)
torch.set_printoptions(precision=4)
torch.backends.cudnn.benchmark = True
torch.set_printoptions(sci_mode=False)

the rosdep view is empty: call 'sudo rosdep init' and 'rosdep update'


Using Device 1 : TITAN Xp


# Set Arguments

In [2]:
import argparse
import sys
import os
import time
import pickle

parser = argparse.ArgumentParser()

'''Training Parameters'''
parser.add_argument('--batch_size', type=int, default=300, help='minibatch size')
parser.add_argument('--num_epochs', type=int, default=200, help='number of epochs')
parser.add_argument('--grad_clip', type=float, default=5., help='clip gradients at this value')
parser.add_argument('--learning_rate', type=float, default=0.001, help='learning rate')
parser.add_argument('--learning_rate_clip', type=float, default=0.0000001, help='learning rate clip')
parser.add_argument('--decay_rate', type=float, default=.85, help='decay rate for rmsprop')
parser.add_argument('--weight_decay', type=float, default=.0001, help='decay rate for rmsprop')
parser.add_argument('--batch_norm_decay', type=float, default=.999, help='decay rate for rmsprop')
parser.add_argument('--keep_prob', type=float, default=1.0, help='dropout keep probability')
parser.add_argument('--lamda_weights', type=float, default=0.01, help='weight of rotation error')
parser.add_argument('--data_argumentation', type=bool, default=True, help='whether do data argument')
parser.add_argument('--is_normalization', type=bool, default=True, help='whether do data normalization')
parser.add_argument('--target_image_size', default=[300, 300], nargs=2, type=int, help='Input images will be resized to this for data argumentation.')

'''Configure'''
parser.add_argument('--network', type=str, default='vggnet_localization')
parser.add_argument('--model_dir', type=str, default='/notebooks/global_localization/dual_resnet_torch', help='rnn, gru, or lstm')


parser.add_argument('--train_dataset', type=str, default = ['/notebooks/michigan_nn_data/2012_01_08',
                                                            '/notebooks/michigan_nn_data/2012_01_15',
                                                            '/notebooks/michigan_nn_data/2012_01_22',
                                                            '/notebooks/michigan_nn_data/2012_02_02',
                                                            '/notebooks/michigan_nn_data/2012_02_04',
                                                            '/notebooks/michigan_nn_data/2012_02_05',
                                                            '/notebooks/michigan_nn_data/2012_03_31',
                                                            '/notebooks/michigan_nn_data/2012_09_28'])

'''
#parser.add_argument('--train_dataset', type=str, default = ['/notebooks/michigan_nn_data/test'])
parser.add_argument('--train_dataset', type=str, default = ['/notebooks/michigan_nn_data/2012_01_08'])
'''
parser.add_argument('--norm_tensor', type=str, default = ['/notebooks/global_localization/norm_mean_std.pt'])

parser.add_argument('--seed', default=1337, type=int)
parser.add_argument('--save_every', type=int, default=3000, help='save frequency')
parser.add_argument('--display', type=int, default=20, help='display frequency')
parser.add_argument('--tensorboard', type=bool, default=True, help='open tensorboard')
parser.add_argument('--cuda_device', type=int, default=1, help='cuda device')

sys.argv = ['']
args = parser.parse_args()

# Load Dataset

In [3]:
from torch.utils.data import Dataset, DataLoader, TensorDataset
import torchvision.transforms as transforms
import tf.transformations as tf_tran
from tqdm import tqdm
from PIL import Image
import numpy as np
import random

import gpytorch

import torch.nn as nn
import torch.optim as optim
from torchlib import resnet, vggnet
from torchlib.cnn_auxiliary import normalize, denormalize_navie, denormalize, get_relative_pose, translational_rotational_loss
from torchlib.utils import LocalizationDataset, display_loss, data2tensorboard
import time

transform = transforms.Compose([transforms.ToTensor()])
dataset = LocalizationDataset(dataset_dirs = args.train_dataset, \
                              image_size = args.target_image_size, \
                              transform = transform, get_pair = False, sampling_rate=2)

[args.norm_mean, args.norm_std] = torch.load(*args.norm_tensor)
print('Load norm and std:',*args.norm_tensor)

dataloader = DataLoader(dataset, batch_size=args.batch_size, \
                        shuffle=True, num_workers=0, \
                        drop_last=True, pin_memory=True)

100%|██████████| 1403/1403 [00:02<00:00, 689.08it/s]

Load norm and std: /notebooks/global_localization/norm_mean_std.pt


# Define Model

In [4]:
from torch.cuda.amp import autocast, GradScaler
from torchlib.GPs import Backbone, NN, BaseModule

class NN(nn.Module):
    def __init__(self):
        super().__init__()
        self.global_context = vggnet.vggnet(input_channel=2048,opt="context")
        self.global_regressor = vggnet.vggnet(opt="regressor")
        
    def forward(self,input_data):
        context_feat = self.global_context(input_data)
        #context_feat = input_data
        output,feature_t, feature_r = self.global_regressor(context_feat)
        return output, feature_t, feature_r
        
class Model(nn.Module):
    def __init__(self):
        super().__init__()
        self.backbone = Backbone()
        self.nn = NN()
        
    def forward(self, input_data):
        dense_feat = self.backbone(input_data)
        #dense_feat = input_data
        output, _, _ = self.nn(dense_feat)
        trans_pred, rot_pred = torch.split(output, [3, 4], dim=1)
        return trans_pred, rot_pred
        
class PosePredictor(BaseModule):
    def __init__(self, norm_mean, norm_std, args, is_training=True, mixed_precision=True):
        super().__init__(norm_mean, norm_std, args)
        self.model = Model().to(self.device)
        
        self.disable_requires_grad(self.model.backbone)
        #self.disable_requires_grad(self.model.nn.global_context)
        
        self.mixed_precision = mixed_precision
        if self.mixed_precision:
            self.scaler = GradScaler()
        if is_training:
            self.optimizer = optim.Adam(self._optimize())
            self.scheduler = optim.lr_scheduler.LambdaLR(optimizer=self.optimizer,
                                                             lr_lambda=lambda epoch: args.decay_rate**epoch)
        else:
            self.disable_requires_grad(self.model)
    
    def _optimize(self):
        # GP
        optimizer = [
                {'params': self.model.nn.global_regressor.parameters(), \
                 'lr': args.learning_rate,'weight_decay':args.weight_decay},
                {'params': self.model.nn.global_context.parameters(), \
                 'lr': args.learning_rate * 0.1,'weight_decay':args.weight_decay}]
                
        return optimizer
    
    def train(self, x, y):
        # Step 0: zero grad
        self.optimizer.zero_grad()
        
        start = time.time()
        # Step 1: get data
        x,y = x.to(self.device),y.to(self.device)
        if args.is_normalization:
            y = normalize(y,self.norm_mean, self.norm_std)
            
        # Step 2: training
        assert trainer.model.training == True
        if self.mixed_precision:
            with autocast():
                single_loss = self._loss(x, y)
            self.scaler.scale(single_loss).backward()
            self.scaler.step(self.optimizer)
            self.scaler.update()
        else:
            single_loss = self._loss(x, y)
            single_loss.backward()
            self.optimizer.step()
            
        batch_time = time.time() - start
        
        return float(single_loss), batch_time
    
    def _loss(self, x, y):

        trans_pred, rot_pred = self.model(x)
        trans_gt, rot_gt = torch.split(y, [3, 4], dim=1)
    
        #trans_loss = nn.functional.mse_loss(input=trans_pred, target=trans_gt)
        trans_loss = torch.sum((trans_pred - trans_gt)**2,dim=1).mean()
        
        rot_loss = 1. - torch.mean(torch.square(torch.sum(torch.mul(rot_pred,rot_gt),dim=1)))

        loss = trans_loss + args.lamda_weights * rot_loss
    
        return loss
    
    def eval_forward(self,x,y,output_denormalize = True):
        # Step 1: get data
        x,y = x.to(self.device),y.to(self.device)
        
        # Step 2: forward
        assert trainer.model.training == False
        
        trans_pred, rot_pred = self.model(x)

        if args.is_normalization and output_denormalize:
            trans_pred = denormalize_navie(trans_pred, self.norm_mean, self.norm_std)
            
        # Step 3: split output
        trans_gt, rot_gt = torch.split(y, [3, 4], dim=1)
        return trans_pred, rot_pred, trans_gt, rot_gt

In [5]:
trainer = PosePredictor(args.norm_mean,args.norm_std,args,mixed_precision=True)
'''
state_dict = torch.load(os.path.join(trainer.args.model_dir, 'pretrained.pth'))
for key in list(state_dict):
    if 'resnet' not in key:
        state_dict.pop(key)
trainer.model.backbone.load_state_dict(state_dict)

state_dict = torch.load(os.path.join(trainer.args.model_dir, 'pretrained.pth'))
for key in list(state_dict):
    if 'resnet' in key:
        state_dict.pop(key)
trainer.model.nn.load_state_dict(state_dict)
'''
#trainer.load_model('pretrained_feature.pth')
#trainer.load_model('pretrained_nn.pth')
trainer.load_model('pretrained_cnn.pth')
#trainer.load_model('pretrained_old.pth',strict = False)
trainer.show_require_grad()

Successfully loaded model to TITAN Xp.
nn.global_context.context.squeeze.0.weight torch.Size([128, 2048, 1, 1])
nn.global_context.context.squeeze.0.bias torch.Size([128])
nn.global_context.context.context5_1.0.weight torch.Size([128, 128, 3, 3])
nn.global_context.context.context5_1.0.bias torch.Size([128])
nn.global_context.context.context5_2.0.weight torch.Size([128, 128, 3, 3])
nn.global_context.context.context5_2.0.bias torch.Size([128])
nn.global_context.context.context5_3.0.weight torch.Size([128, 128, 3, 3])
nn.global_context.context.context5_3.0.bias torch.Size([128])
nn.global_context.context.context5_4.0.weight torch.Size([128, 128, 3, 3])
nn.global_context.context.context5_4.0.bias torch.Size([128])
nn.global_context.context.squeeze2.0.weight torch.Size([64, 128, 1, 1])
nn.global_context.context.squeeze2.0.bias torch.Size([64])
nn.global_regressor.regressor.fc1_trans.0.weight torch.Size([4096, 6400])
nn.global_regressor.regressor.fc1_trans.0.bias torch.Size([4096])
nn.global_

# Training

In [6]:
if args.tensorboard:
    import os
    os.system('rm -rf runs/nn')
    from torch.utils.tensorboard import SummaryWriter
    writer = SummaryWriter('runs/nn')
    time.sleep(3)

In [7]:
trainer.model.train()
for e in range(args.num_epochs):
    minibatch_iter = tqdm(dataloader)
    train_loss = 0.
    for b, data in enumerate(minibatch_iter):
        x,y = data.values()

        loss, batch_time = trainer.train(x,y)
        
        train_loss += loss
        ave_loss = train_loss/(b+1)
        step = e*len(dataloader)+(b+1)
        # display data
        minibatch_iter.set_postfix(ave = ave_loss, loss=loss,lr=trainer.scheduler.get_last_lr()[0])
        # tensorboard
        trainer.data2tensorboard(writer,'training loss',{'item loss':loss,'batch loss':ave_loss},step)
        # save model
        #trainer.save_model_step(e,step)
        # step scheduler
        trainer.schedule_step(step,50)

 38%|███▊      | 82/213 [02:47<04:28,  2.05s/it, ave=0.00326, loss=0.0034, lr=2.8e-5]  


KeyboardInterrupt: 

In [8]:
trainer.save_model('pretrained_cnn.pth')

Saving model to pretrained_cnn.pth


# Evaluation

In [6]:
trainer.model.eval()
for e in range(args.num_epochs):
    minibatch_iter = tqdm(dataloader)
    train_loss = 0.
    for b, data in enumerate(minibatch_iter):
        x,y = data.values()
        break
    break
    
with torch.no_grad():
    trans_pred, rot_pred, trans_gt, rot_gt = trainer.eval_forward(x,y)

  0%|          | 0/42 [00:00<?, ?it/s]


In [7]:
trans_pred

tensor([[  -308.7205,    328.2235,    -11.3248],
        [    -5.6888,    240.6147,     -5.5175],
        [   -30.2300,     38.2000,     -2.8123],
        [  -308.3470,    480.1413,    -12.0075],
        [    22.8371,    239.6704,     -4.3876],
        [   -72.8947,    285.4908,     -9.1210],
        [    23.8033,    344.5584,     -1.9748],
        [  -180.4330,    699.0442,    -12.2002],
        [   -27.4269,    179.3230,     -3.6110],
        [   -83.7531,    473.6189,    -10.7122],
        [    34.9525,    464.3186,     -2.3831],
        [  -297.1191,    545.1538,    -12.5989],
        [  -189.8914,    391.9798,    -14.7914],
        [    23.0532,    390.3834,     -2.0233],
        [  -279.9417,    687.1698,    -14.2649],
        [    29.4659,    416.6886,     -1.9440],
        [  -306.0102,    449.9053,    -12.2469],
        [  -176.1054,    497.9644,    -12.1934],
        [   -49.9920,    454.1875,    -10.4992],
        [   -57.4626,    157.1786,     -3.2896],
        [  -311.0583

In [8]:
trans_gt

tensor([[  -307.5341,    321.5056,    -11.2931],
        [   -13.3284,    238.4964,     -5.6624],
        [   -31.1468,     39.1289,     -2.8683],
        [  -310.0840,    480.3721,    -11.9552],
        [    23.5332,    240.1028,     -4.3579],
        [   -73.2721,    284.1154,     -9.1723],
        [    24.4777,    344.3256,     -1.9789],
        [  -180.3148,    700.2698,    -12.2590],
        [   -22.8906,    173.7944,     -3.5708],
        [   -83.9261,    473.8421,    -10.7335],
        [    35.0666,    463.8519,     -2.3491],
        [  -295.8101,    546.9922,    -12.5986],
        [  -189.5962,    393.3441,    -14.8420],
        [    23.0018,    388.8492,     -1.9940],
        [  -280.8733,    686.4551,    -14.2578],
        [    26.5253,    416.3970,     -2.0472],
        [  -305.0526,    450.3047,    -12.2457],
        [  -173.8248,    496.1037,    -12.0962],
        [   -52.6341,    453.5656,    -10.4483],
        [   -58.5504,    156.6659,     -3.2581],
        [  -310.5334

new: 7-21  0.0019

new: 5-15  0.0013

new: 3-11 0.0010

old: 8-25  0.0021

In [9]:
nn.functional.mse_loss(input=trans_pred, target=trans_gt)

tensor(3.6761, device='cuda:1')

In [10]:
torch.sum((trans_pred - trans_gt)**2,dim=1).mean()

tensor(11.0284, device='cuda:1')

In [11]:
rot_pred

tensor([[ 0.7289,  0.6845,  0.0046,  0.0109],
        [ 0.0168,  0.9998,  0.0070, -0.0052],
        [ 0.9840, -0.1773, -0.0137,  0.0072],
        ...,
        [ 0.6659, -0.7460, -0.0111,  0.0043],
        [-0.4172,  0.9088,  0.0054, -0.0011],
        [ 0.6819,  0.7314,  0.0057,  0.0092]], device='cuda:1')

In [12]:
rot_gt

tensor([[ 0.7119,  0.7016,  0.0061,  0.0290],
        [ 0.0261,  0.9996, -0.0083, -0.0070],
        [ 0.9930, -0.1084, -0.0411,  0.0236],
        ...,
        [-0.6521,  0.7579, -0.0161,  0.0121],
        [-0.4887,  0.8721,  0.0181,  0.0181],
        [ 0.7001,  0.7137,  0.0224, -0.0015]], device='cuda:1')

In [13]:
1. - torch.mean(torch.square(torch.sum(torch.mul(rot_pred,rot_gt),dim=1)))

tensor(0.0113, device='cuda:1')